# Summarizing documents with Kernel Memory

This notebook shows how to leverage Kernel Memory pipelines to generate custom information such as a summaries.

In KM terms, a Summary is a Synthetic Memory, which means it's some information derived from other memories. KM includes one example of Synthetic Memory generation, in the form of a Summarization Handler, that users can optionally use when ingesting documents. In fact, considering this handler, there are at least three ways to import a document:

1. Import a document, creating memory records for each "chunk".
2. Import a document, creating memory records for each "chunk", and generating a summary memory record.
3. Import a document, creating only the summary synthetic memory record.

If you add your custom pipeline handlers to KM, you can further customize what KM does with your documents. For instance you could:

1. Use a Translation Handler to translate every document into other languages
2. Use a Monitoring Handler to analyze memories and create or update some statistics stored in memory (or anywhere you like)
3. Use a ChatLog Handler to detect customers having troubles with some product
4. etc.

Let's see how to create and retrieve summaries.

## Configuration

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with your OpenAI API Key.

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> ```

In [1]:
#r "nuget: dotenv.net, 3.1.3"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

Let's setup KM as usual:

In [2]:
#r "nuget: Microsoft.KernelMemory.Core, 0.23.231218.1"

using Microsoft.KernelMemory;

var memory = new KernelMemoryBuilder()
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .Build<MemoryServerless>();

Console.WriteLine("Kernel Memory ready.");

Installed Packages Microsoft.KernelMemory.Core, 0.23.231218.1

info: Microsoft.KernelMemory.Handlers.TextExtractionHandler[0]
      Handler 'extract' ready
info: Microsoft.KernelMemory.Handlers.TextPartitioningHandler[0]
      Handler 'partition' ready
info: Microsoft.KernelMemory.Handlers.SummarizationHandler[0]
      Handler 'summarize' ready
info: Microsoft.KernelMemory.Handlers.GenerateEmbeddingsHandler[0]
      Handler 'gen_embeddings' ready, 1 embedding generators
info: Microsoft.KernelMemory.Handlers.SaveRecordsHandler[0]
      Handler save_records ready, 1 vector storages
info: Microsoft.KernelMemory.Handlers.DeleteDocumentHandler[0]
      Handler 'private_delete_document' ready
info: Microsoft.KernelMemory.Handlers.DeleteIndexHandler[0]
      Handler 'private_delete_index' ready
info: Microsoft.KernelMemory.Handlers.DeleteGeneratedFilesHandler[0]
      Handler 'delete_generated_files' ready
Kernel Memory ready.


# Import document with custom pipeline steps

Now, let's import and summarize a document.

Note the `steps` parameter, that we use to customize the pipeline, asking to just generate a summary record, nothing else. 
The constant is just a shortcut for a list of strings: `{ "extract", "summarize", "gen_embeddings", "save_records" }`.

In [3]:
await memory.ImportDocumentAsync("NASA-news.pdf", documentId: "doc001", steps: Constants.PipelineOnlySummary);

Console.WriteLine("Document imported and summarized.");

Document imported and summarized.


# Fetch and print the summary

In [4]:
var results = await memory.SearchSummariesAsync(filter: MemoryFilters.ByDocument("doc001"));

foreach (var result in results)
{
    var summary = result.Partitions.First().Text;
    Console.WriteLine($"== {result.SourceName} summary ==\n\n{summary}\n");
}

== NASA-news.pdf summary ==

NASA is inviting media to see the test version of the Orion spacecraft and the recovery hardware that will be used for the Artemis II mission. The event will take place at Naval Base San Diego on August 2nd. Personnel from NASA, the U.S. Navy, and the U.S. Air Force will be available to speak with media. The recovery operations team is currently conducting tests in the Pacific Ocean to prepare for Artemis II, which will be NASA's first crewed mission under the Artemis program. The Artemis II crew will participate in recovery testing at sea next year.

